<a href="https://colab.research.google.com/github/mdkennedy3/Useful_Robotics_Scripts/blob/master/Budget_Sankey_Diagram_from_bank_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Black in Robotics Sankey Diagram for FY 2022 Budget
<img src="https://images.squarespace-cdn.com/content/v1/5f7e2c5f6b242d64718ffde9/48f012cc-68d3-46b6-acf3-f5a6bc7da8e5/BiR_Logo_primary_vector_graphic+cropped.png?format=1500w" alt="drawing" width="200"/>

This diagram shows the flow of Revenue and Expenses this fiscal year 2022.

First read in bank csv file for FY 2022 transactions


In [1]:
import csv
from google.colab import files
import numpy as np
import os
import pandas as pd #https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html
import plotly.graph_objects as go #https://www.python-graph-gallery.com/sankey-diagram-with-python-and-plotly 
from tables import expression
!pip install -U kaleido #for downloading figures


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79.9 MB 1.2 MB/s 


In [2]:
#save .csv to your local machine, then select the csv file for upload when running this block: (for FY 2022) https://drive.google.com/file/d/1QJ5mg0HM_NKNwV_vjEryb6a5grJ7Y2zK/view?usp=share_link 
uploaded = files.upload()
os.listdir()

Saving quick_books_FY2022.csv to quick_books_FY2022.csv


['.config', 'quick_books_FY2022.csv', 'sample_data']

In [3]:
bank_trans = pd.read_csv('quick_books_FY2022.csv') #read in the csv file
bank_trans #display csv file contents

,DATE,DESCRIPTION,AMOUNT,Assigned To,Category
0,12/14/2022,Paypal,$154.00,Added to: Deposit: Services 12/14/2022 $154.00,BiR Store
1,12/09/2022,PayPal,-$1.65,Added to: Expense: Owner's Pay & Personal Exp...,Paypal transaction expense
2,12/05/2022,PayPal,-$0.84,Added to: Expense: Owner's Pay & Personal Exp...,Paypal transaction expense
3,12/05/2022,PayPal,-$0.84,Added to: Expense: Owner's Pay & Personal Exp...,Paypal transaction expense
4,12/05/2022,PayPal,-$0.84,Added to: Expense: Owner's Pay & Personal Exp...,Paypal transaction expense
...,...,...,...,...,...
57,03/07/2022,USPS,-$18.20,Added to: Expense: Office Supplies & Software...,Operation Expense
58,03/03/2022,Intuit,-$180.00,Added to: Expense: Office Supplies & Software...,Operation Expense
59,02/14/2022,Hopper Dean Foundation Wire Transfer,"$100,000.00",Added to: Payment: Accounts Receivable (A/R) ...,Partner Donation
60,01/31/2022,Intuit,-$180.00,Added to: Expense: Office Supplies & Software...,Operation Expense


Now separate into expenses vs revenue

In [30]:
class Generate_Sankey(object):
  """
  This class takes in a CSV file that has the following headers: 
  DATE: date of transaction XX/XX/XXXX	
  DESCRIPTION: discription of revenue/expense (e.g. source identity)
  AMOUNT: $ amount of transaction
  Category: This must be done, categories for type of expense (e.g. operational vs donation vs chapter expense), the script will look at the sign of the transaction 
  and determine if its an expense or revenue automatically

  Note: this plot does not just do Revenue and Expenses, it also shows how much steady funds there are, hence one line of the CSV file should have a Category called "Initial Year Balance"
  """
  def __init__(self,bank_trans=None, init_balance_name="Initial Year Balance", expense_commitment=None):
    '''
    bank_trans: dataframe object for CSV bank data
    init_balance_name: string for name in category for initial balance included in CSV
    expense_commitment: Dictionary for name of commitment and dollar amount (e.g. {"Google Award":75000, "Travel Award":2000})
    '''
    self.bank_trans = bank_trans 
    self.init_balance_name= init_balance_name
    if type(expense_commitment) != None: 
      self.expense_commitment = expense_commitment #if commitments were specified, then save for usage
      self.net_expense_commitment = np.sum([expense_commitment[key] for key in expense_commitment.keys()]) #sum the commitments
    self.bank_trans["AMOUNT"] = self.bank_trans["AMOUNT"].replace('[\$,]', '', regex=True).astype(float) #remove $ sign and commas from dollar amount, and make it a float
    #sort into Expense vs Revenue
    self.net_expenses = self.bank_trans[self.bank_trans["AMOUNT"] < 0]
    self.net_revenue = self.bank_trans[self.bank_trans["AMOUNT"] > 0]
    #find the unique categories:
    self.expense_categories = self.net_expenses["Category"].unique().tolist()
    self.revenue_categories = self.net_revenue["Category"].unique().tolist()
    #empty dictionaries to populate expense/revenue
    self.expense_cat_values = dict(zip(self.expense_categories, ([] for _ in self.expense_categories))) 
    self.revenue_cat_values = dict(zip(self.revenue_categories, ([] for _ in self.revenue_categories)))
    # sum the expense and revenue categories:
    for category in self.expense_categories:  
      self.expense_cat_values[category] = np.sum(self.net_expenses.loc[self.net_expenses["Category"]==category,["AMOUNT"]]["AMOUNT"].tolist())
    for category in self.revenue_categories:  
      self.revenue_cat_values[category] = np.sum(self.net_revenue.loc[self.net_revenue["Category"]==category,["AMOUNT"]]["AMOUNT"].tolist())
    # Set the other values:
    self.init_account_balance = self.net_revenue.loc[self.net_revenue["Category"]==init_balance_name,"AMOUNT"].tolist()[0] #the initial balance in the account at the beginning of the year (expected name is init_balance_name specified above)
    self.net_expense_value_actuals = np.sum([self.expense_cat_values[key] for key in self.expense_cat_values.keys()]) #net Expenses
    if type(expense_commitment) != None:
      self.net_expense_value = self.net_expense_value_actuals+ self.net_expense_commitment #If expense commitments are present, then add them here
    else:
      self.net_expense_value = self.net_expense_value_actuals
    #Balance/Revenue
    self.net_input_balance = np.sum([self.revenue_cat_values[key] for key in self.revenue_cat_values.keys()]) #net Revenue + initial balance
    self.net_revenue_value = self.net_input_balance - self.init_account_balance #This is strictly revenue (not the balance coming in)  
    #Reserves (Net balance minus net expenses)
    self.reserves = self.net_input_balance + self.net_expense_value
    #Current balance in account:
    self.current_account_balance = self.net_input_balance + self.net_expense_value_actuals
    if type(expense_commitment) != None:
      self.ordered_list_of_central_nodes = ['Revenue','Initial Balance','Net Input Balance', 'Reserve', 'Net Expense', 'Expense Actuals', 'Expense Commits']
      nodes_value_list = [self.net_revenue_value, self.init_account_balance, self.net_input_balance, self.reserves, self.net_expense_value,self.net_expense_value_actuals,self.net_expense_commitment]
    else:
      self.ordered_list_of_central_nodes = ['Revenue','Initial Balance','Net Input Balance', 'Reserve', 'Net Expense', 'Expense Actuals']
      nodes_value_list = [self.net_revenue_value, self.init_account_balance, self.net_input_balance, self.reserves, self.net_expense_value,self.net_expense_value_actuals]
    #Zip the dictionary
    self.central_nodes_dict = dict(zip(self.ordered_list_of_central_nodes,nodes_value_list))

  def generate_sankey_objects_low_level(self):
    """This function, generates the objects for the Sankey plot including the individual transactions"""
    return

  def generate_sankey_by_categories(self):
    """
    This function returns the Sankey plots by categories (not small transactions)
    Sources are all the a) revenue categories (which point to net revenue) 
    b) then consistent structure for i) init balance, revenue which go to ii) Net Input Balance (single) which then points to iii) Expense and reserve iv) and Expense then points to expense_actuals
    """
    self.sankey_node_labels = [] #these are all nodes labels (source/target) [this is both the name of the node as well as teh dollar amount]
    self.sankey_node_values = [] #these are the monetoary values each node holds
    self.sankey_target_list = [] #these are the target node values

    #Step 1: specify the source list (nodes) 
    self.sankey_source_list = []
    #First iterate through the revenue categories, each of these
    idx = 0 #node counter
    for rev_cat in self.revenue_cat_values.keys():
      if rev_cat != self.init_balance_name:
        disp_name= rev_cat + " $" + str(self.revenue_cat_values[rev_cat]) #name of the category and dollar amount as a string
        self.sankey_node_labels.append(disp_name)
        self.sankey_node_values.append(self.revenue_cat_values[rev_cat])
        self.sankey_source_list.append(idx)
        idx += 1 #add nodes from counter
        # all of these point to the same Revenue node
        self.sankey_target_list.append(len([key for key in self.revenue_cat_values.keys()])-1) #ensure they all point to the index that is +1 more greater than teh length of the key list (as we make revenue that index)

    #Step 2: now that all the revenue categories are added, and they point to the node that is one greater than the list length with just revenue categories, so now we add Revenue to the target label, value and source list
    disp_name = "Revenue"+" $" + str(self.central_nodes_dict["Revenue"])
    self.sankey_node_labels.append(disp_name)
    self.sankey_node_values.append(self.central_nodes_dict["Revenue"])
    self.sankey_source_list.append(idx) #(idx should be equal right now to len([key for key in self.revenue_cat_values.keys()]))
    self.sankey_target_list.append(len([key for key in self.revenue_cat_values.keys()])+1) #hop two so we can connect init balance
    idx += 1
    
    #Step 3: now connect Init Balance
    disp_name = "Initial Balance"+" $" + str(self.central_nodes_dict["Initial Balance"])
    self.sankey_node_labels.append(disp_name)
    self.sankey_node_values.append(self.central_nodes_dict["Initial Balance"])
    self.sankey_source_list.append(idx) #(idx should be equal right now to len([key for key in self.revenue_cat_values.keys()]))
    self.sankey_target_list.append(len([key for key in self.revenue_cat_values.keys()])+1) #hop one so we can connect init balance
    idx += 1
    
    #Step 4: Now connect Balance as a source/label/value and have it point to two outputs Expense & Reserve
    disp_name = "Net Income Balance"+" $" + str(self.central_nodes_dict["Net Input Balance"])
    self.sankey_node_labels.append(disp_name)
    # self.sankey_node_values.append(self.central_nodes_dict["Net Input Balance"])
    self.sankey_source_list.append(idx) #Make this a source, but it should connect to the following two nodes
    self.sankey_target_list.append(idx+1) #hop one so we can connec to Reserve

    self.sankey_source_list.append(idx) #Make this a source, but it should connect to the following two nodes
    self.sankey_target_list.append(idx+2) #hop one so we can connect Expense
    idx += 1
    
    #Step 5: Now add Reserve
    disp_name = "Reserve"+" $" + str(self.central_nodes_dict["Reserve"])
    self.sankey_node_labels.append(disp_name)
    self.sankey_node_values.append(np.abs(self.central_nodes_dict["Reserve"]))
    # self.sankey_target_list.append(idx) #this is just a sink not a source for the FY
    idx += 1
    
    #Step 6: Now add Expense
    disp_name = "Net Expense"+" $" + str(self.central_nodes_dict["Net Expense"])
    self.sankey_node_labels.append(disp_name)
    self.sankey_node_values.append(np.abs(self.central_nodes_dict["Net Expense"])) #the value here must be a positive quantity for the graph to work
    self.sankey_source_list.append(idx) #Make this a source, but it should connect to the following two nodes
    self.sankey_target_list.append(idx+1) #hop one 
    self.sankey_source_list.append(idx) #Make this a source, but it should connect to the following two nodes
    self.sankey_target_list.append(idx+2) #hop two
    idx += 1    

    #Step 7: Now add Expenditures Committed (but keep track of index to back track for Expense Actuals); Add all the categories of committed
    disp_name = "Expenditures Committed"+" $" + str(self.central_nodes_dict["Expense Commits"])
    self.sankey_node_labels.append(disp_name)
    self.sankey_node_values.append(np.abs(self.central_nodes_dict["Expense Commits"])) 
    idx += 1


    #Step 8: Now add Expendutres Actuals; now add all the categories of expense actuals
    disp_name = "Expense Actuals"+" $" + str(self.central_nodes_dict["Expense Actuals"])
    self.sankey_node_labels.append(disp_name)
    self.sankey_node_values.append(np.abs(self.central_nodes_dict["Expense Actuals"])) 
    idx_actuals = idx #sub expenditrues need to come from this
    idx += 1

    #prototype
    # disp_name = "Test subexpense"+" $100" 
    # self.sankey_node_labels.append(disp_name)
    # self.sankey_node_values.append(100) 
    # self.sankey_source_list.append(idx_actuals)
    # self.sankey_target_list.append(idx)
    # idx += 1


    # disp_name = "Test subexpense 2"+" $50" 
    # self.sankey_node_labels.append(disp_name)
    # self.sankey_node_values.append(50) 
    # self.sankey_source_list.append(idx_actuals)
    # self.sankey_target_list.append(idx)
    # idx += 1

    # #now add all the subcategories to Expense Actuals:
    for exp_cat in self.expense_cat_values.keys():
      disp_name= exp_cat + " $" + str(self.expense_cat_values[exp_cat]) #name of the category and dollar amount as a string
      self.sankey_node_labels.append(disp_name)
      self.sankey_node_values.append(np.abs(self.expense_cat_values[exp_cat]))
      self.sankey_source_list.append(idx_actuals)
      self.sankey_target_list.append(idx) #ensure they all point to the index that is +1 more greater than teh length of the key list (as we make revenue that index)
      idx += 1 #add nodes from counter
     

    return 


  def plot_sankey(self):

    link = dict(source = self.sankey_source_list, target = self.sankey_target_list, value = self.sankey_node_values, label=self.sankey_node_labels)#, color=color)
    node = dict(pad = 15, thickness = 15, line = dict(color = "black", width = 0.5), label=self.sankey_node_labels)
    data = go.Sankey(link = link, node=node)

    fig = go.Figure(data)

    fig.show()
    fig.update_layout(title_text="BiR Budget for 2022",
                  font_size=16)
    #Write this as an html so that it can be opened then PNG snapshot made
    filename = "BiR_flowchart.html"
    go.Figure.write_html(fig,filename)
    files.download(filename)
    return


The following cell must be update each year to match the current years banking details CSV, and expense commitments

In [31]:
expense_commitment = {"Google Award":-75000, "Travel Award":-2000, "Amazon BiR Boston Chapter":-40000} #Specify the financial commitments (that are outstanding)
cls_obj = Generate_Sankey(bank_trans=bank_trans, expense_commitment=expense_commitment)
# print(cls_obj.expense_categories)
# print(cls_obj.expense_cat_values)
print("Net expense ",cls_obj.net_expense_value)
print("Net expense values actuals", cls_obj.net_expense_value_actuals)
print("net committment: ",cls_obj.net_expense_commitment)
print("Net revenue ",cls_obj.net_revenue_value)
print("net input balance", cls_obj.net_input_balance)
print("init balance: ", cls_obj.init_account_balance)
print("Reserves ",cls_obj.reserves)
print("end of year balance (without commitments)",cls_obj.current_account_balance)
#display key values:
cls_obj.central_nodes_dict

Net expense  -128672.07
Net expense values actuals -11672.07
net committment:  -117000
Net revenue  225633.94999999998
net input balance 312824.48
init balance:  87190.53
Reserves  184152.40999999997
end of year balance (without commitments) 301152.41


{'Revenue': 225633.94999999998,
 'Initial Balance': 87190.53,
 'Net Input Balance': 312824.48,
 'Reserve': 184152.40999999997,
 'Net Expense': -128672.07,
 'Expense Actuals': -11672.07,
 'Expense Commits': -117000}

In [32]:
cls_obj.generate_sankey_by_categories()
cls_obj.plot_sankey()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
print("Labels: ",cls_obj.sankey_node_labels)
print("values: ",cls_obj.sankey_node_values)
print("sources: ",cls_obj.sankey_source_list)
print("targets: ",cls_obj.sankey_target_list)
cls_obj.expense_cat_values

Labels:  ['BiR Store $154.0', 'Partner Donation $225000.0', 'Reimbursement and Misc $104.95', 'BiR Workshop Registration fee $375.0', 'Revenue $225633.94999999998', 'Initial Balance $87190.53', 'Net Income Balance $312824.48', 'Reserve $184152.40999999997', 'Net Expense $-128672.07', 'Expenditures Committed $-117000', 'Expense Actuals $-11672.07', 'Paypal transaction expense $-4.17', 'Operation Expense $-3090.2', 'Bank Fee $-229.0', 'BIR Workshop Honorarium $-700.0', 'Chapter Expense $-2123.49', 'BiR Workshop Kit and Shipping $-1838.21', 'Resume Book Promotion  $-375.0', 'Refund Loan $-3312.0']
values:  [154.0, 225000.0, 104.95, 375.0, 225633.94999999998, 87190.53, 184152.40999999997, 128672.07, 117000, 11672.07, 4.17, 3090.2, 229.0, 700.0, 2123.49, 1838.21, 375.0, 3312.0]
sources:  [0, 1, 2, 3, 4, 5, 6, 6, 8, 8, 10, 10, 10, 10, 10, 10, 10, 10]
targets:  [4, 4, 4, 4, 6, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


{'Paypal transaction expense': -4.17,
 'Operation Expense': -3090.2,
 'Bank Fee': -229.0,
 'BIR Workshop Honorarium': -700.0,
 'Chapter Expense': -2123.49,
 'BiR Workshop Kit and Shipping': -1838.21,
 'Resume Book Promotion ': -375.0,
 'Refund Loan': -3312.0}